In [106]:
import pandas as pd
import numpy as np
import ast
from typing import Dict, List

pd.set_option("display.precision", 6)
pd.set_option("display.max_columns", None)
pd.options.mode.chained_assignment = None

# Preproccessing

## FIGHTERS preproccessing

In [133]:
fighters_df = pd.read_csv("data/0.fighters_raw.csv", index_col=0)
fighters_df["dateOfBirth"] = pd.to_datetime(fighters_df["dateOfBirth"])
fighters_df["birthYear"] = fighters_df["dateOfBirth"].dt.year
fighters_cols = [
    "id",
    "name",
    "weight",
    "height",
    "armSpan",
    "legSwing",
    "weightCategory.id",
    "weightCategory.name",
    "dateOfBirth",
    "birthYear",
    "country",
    "city",
    "timezone",
]
fighters_df = fighters_df[fighters_cols]
fighters_df.set_index("id", inplace=True)
fighters_df.head(5)

,name,weight,height,armSpan,legSwing,weightCategory.id,weightCategory.name,dateOfBirth,birthYear,country,city,timezone
id,,,,,,,,,,,,
1,Tanner Boser,115.67,187.96,190.50,NaN,9,Тяжелый вес,1991-08-02,1991.0,Canada,Bonnyville,America/Edmonton
2,Giacomo Lemos,112.04,190.50,190.50,NaN,9,Тяжелый вес,1989-06-23,1989.0,Brazil,NaN,America/Sao_Paulo
3,Shamil Abdurakhimov,106.59,190.50,193.04,105.41,9,Тяжелый вес,1981-09-02,1981.0,Dagestan,Makhachkala,Europe/Moscow
4,Klidson Abreu,92.99,182.88,187.96,NaN,8,Полутяжелый вес,1992-12-24,1992.0,Brazil,Manaus,America/Manaus
5,Yoshihiro Akiyama,77.11,177.80,190.50,106.68,6,Полусредний вес,1975-07-29,1975.0,Japan,Ikuno,Asia/Tokyo


## Заполняем пропуски в колонках

In [134]:
# Находим все возможные весовые категории
avg_weight_in_weight_category = fighters_df.groupby(by="weightCategory.id").mean()['weight']

# Замена пустых значений веса на среднее по весовой категории
def replace_null_to_avg_weight(row):
    if np.isnan(row['weight']):
        return avg_weight_in_weight_category.iloc[row['weightCategory.id'] - 2]
    return row['weight']

fighters_df['weight'] = fighters_df.apply(
    lambda row: replace_null_to_avg_weight(row),
    axis=1
)

fighters_df['weight'].isna().value_counts()

False    3669
Name: weight, dtype: int64

In [135]:
fighters_df[fighters_df['armSpan'].isna()]

,name,weight,height,armSpan,legSwing,weightCategory.id,weightCategory.name,dateOfBirth,birthYear,country,city,timezone
id,,,,,,,,,,,,
514,Mike Davis,65.75,182.88,NaN,NaN,4,Полулегкий вес,1970-01-01,1970.0,New York,South Cairo,NaN
772,Brodie Farber,77.11,185.42,NaN,NaN,7,Средний вес,1980-07-05,1980.0,United States,NaN,America/New_York
773,Nate Loughran,83.91,187.96,NaN,NaN,7,Средний вес,1980-07-28,1980.0,United States,Rohnert Park,America/New_York
775,Benji Radach,83.91,177.80,NaN,NaN,7,Средний вес,1979-04-05,1979.0,United States,NaN,America/New_York
776,Dan Severn,113.40,187.96,NaN,NaN,9,Тяжелый вес,1958-06-08,1958.0,United States,NaN,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...
3673,Rinat Fakhretdinov,83.91,182.88,NaN,NaN,14,Не определена,1991-09-28,1991.0,NaN,NaN,NaN
3674,Carlston Harris,77.11,182.88,NaN,NaN,14,Не определена,1987-07-09,1987.0,NaN,NaN,NaN
3675,Askar Askar,65.77,170.18,NaN,NaN,14,Не определена,1994-08-19,1994.0,NaN,NaN,NaN


In [136]:
# Формула для нахождения размаха рук
# arm_length = height*0.4
# arm_span = arm_length*2 + armLength/2

# Замена пустых значений размаха рук по вымышленной формуле
def replace_null_to_calc_arm_span(row):
    if np.isnan(row['armSpan']) and row['height']:
        height = row['height']
        arm_length = height*0.4
        arm_span = arm_length*2 + arm_length/2
        return arm_span
    return row['armSpan']

fighters_df['armSpan'] = fighters_df.apply(
    lambda row: replace_null_to_calc_arm_span(row),
    axis=1
)

fighters_df['armSpan'].isna().value_counts()

False    3335
True      334
Name: armSpan, dtype: int64

In [137]:
fighters_df[772:781]

,name,weight,height,armSpan,legSwing,weightCategory.id,weightCategory.name,dateOfBirth,birthYear,country,city,timezone
id,,,,,,,,,,,,
773,Nate Loughran,83.910000,187.96,187.96,NaN,7,Средний вес,1980-07-28,1980.0,United States,Rohnert Park,America/New_York
774,Mike Kyle,92.990000,193.04,195.58,NaN,8,Полутяжелый вес,1980-03-31,1980.0,United States,Boise,America/Boise
775,Benji Radach,83.910000,177.80,177.80,NaN,7,Средний вес,1979-04-05,1979.0,United States,NaN,America/New_York
776,Dan Severn,113.400000,187.96,187.96,NaN,9,Тяжелый вес,1958-06-08,1958.0,United States,NaN,America/New_York
777,Cesar Marscucci,89.994459,NaN,NaN,NaN,14,Не определена,1970-01-01,1970.0,Brazil,NaN,America/Sao_Paulo
778,Josh Stuart,72.570000,175.26,175.26,NaN,6,Полусредний вес,1970-01-01,1970.0,United States,NaN,America/New_York
779,Justin Martin,83.910000,182.88,182.88,NaN,7,Средний вес,1970-01-01,1970.0,United States,NaN,America/New_York
780,Harold Howard,108.860000,187.96,187.96,NaN,9,Тяжелый вес,1970-01-01,1970.0,Canada,NaN,America/Montreal
781,Eldo Xavier Dias,81.650000,180.34,180.34,NaN,7,Средний вес,1964-07-16,1964.0,NaN,NaN,NaN


### Пропуски
- пропуски в росте и весе заполнить средними в весовой категории
- пропуски в размахе рук и ног заполнить средним для похожего роста

## EVENTS

In [108]:
events_df = pd.read_csv("data/0.events_raw.csv", index_col=0)
events_df["eventDate.date"] = pd.to_datetime(events_df["eventDate.date"])
events_df.set_index("id", inplace=True)
events_df.drop(events_df[events_df["completed"] == False].index, inplace=True)
events_df.drop(events_df[events_df["winnerId"].isna()].index, inplace=True)
events_df.drop(columns=["completed", "eventDate.timezone_type", "link"], inplace=True)
events_df.head(5)

,avgOdds,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,fighters,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId
id,,,,,,,,,,,,,,,,
5201,[],Denver,USA,104.0,1993-11-12,Europe/Berlin,1646,1923,"[{'fighterId': 1646, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0
5202,[],Denver,USA,52.0,1993-11-12,Europe/Berlin,1777,1883,"[{'fighterId': 1777, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0
5203,[],Denver,USA,59.0,1993-11-12,Europe/Berlin,1908,1923,"[{'fighterId': 1908, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0
5204,[],Denver,USA,57.0,1993-11-12,Europe/Berlin,1631,1646,"[{'fighterId': 1631, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0
5205,[],Denver,USA,138.0,1993-11-12,Europe/Berlin,1646,1924,"[{'fighterId': 1646, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0


#### Парсим колонку `avgOdds`

In [109]:
def parse_odds(row: pd.Series) -> pd.Series:
    """
    Parse 'avgOdds' column.
    :param row: Row of the events dataframe.
    :return: pd.Series with odds for the 1st and the 2nd fighters.
    """
    avg_odds = row["avgOdds"]
    if avg_odds == "[]" or avg_odds == np.nan:
        return pd.Series([np.nan] * 2)
    avg_odds = ast.literal_eval(avg_odds)
    if avg_odds[0]["fighterId"] == row["fighterId_1"]:
        return pd.Series([f.get("value", np.nan) for f in avg_odds])
    else:
        return pd.Series([f.get("value", np.nan) for f in reversed(avg_odds)])

In [110]:
events_df[["odds_f1", "odds_f2"]] = events_df[
    ["avgOdds", "fighterId_1", "fighterId_2"]
].apply(lambda row: parse_odds(row), axis=1)
events_df.drop(columns="avgOdds", inplace=True)
events_df.head(5)

,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,fighters,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,odds_f1,odds_f2
id,,,,,,,,,,,,,,,,,
5201,Denver,USA,104.0,1993-11-12,Europe/Berlin,1646,1923,"[{'fighterId': 1646, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN
5202,Denver,USA,52.0,1993-11-12,Europe/Berlin,1777,1883,"[{'fighterId': 1777, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0,NaN,NaN
5203,Denver,USA,59.0,1993-11-12,Europe/Berlin,1908,1923,"[{'fighterId': 1908, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0,NaN,NaN
5204,Denver,USA,57.0,1993-11-12,Europe/Berlin,1631,1646,"[{'fighterId': 1631, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0,NaN,NaN
5205,Denver,USA,138.0,1993-11-12,Europe/Berlin,1646,1924,"[{'fighterId': 1646, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN


#### Парсим колонку `fighters`

In [111]:
fighter_stats_keys = [
    "hitsTotal",
    "hitsSuccessful",
    "takedownTotal",
    "takedownSuccessful",
    "submissionAttempts",
    "takeovers",
    "accentedHitsTotal",
    "accentedHitsSuccessful",
    "knockdowns",
    "protectionPassage",
    "hitsHeadTotal",
    "hitsHeadSuccessful",
    "hitsBodyTotal",
    "hitsBodySuccessful",
    "hitsLegsTotal",
    "hitsLegsSuccessful",
    "accentedHitsPositionDistanceTotal",
    "accentedHitsPositionDistanceSuccessful",
    "accentedHitsPositionClinchTotal",
    "accentedHitsPositionClinchSuccessful",
    "accentedHitsPositionParterTotal",
    "accentedHitsPositionParterSuccessful",
]


def get_fighter_stats_cols() -> List[str]:
    """
    Get list of fight stats column names for each fighter.
    :return: List of column names with '_f1' suffix
    for the first fighter and '_f2' suffix for the second.
    """
    fighter_stats_cols = []
    for i in range(1, 3):
        for k in fighter_stats_keys:
            fighter_stats_cols.append(f"{k}_f{i}")
    return fighter_stats_cols


def sum_round_stats(stats: List[Dict[str, int]]) -> List[int]:
    """
    Sum stats for a fighter for all rounds of a fight.
    :param stats: List with stats from object of 'fighters' column.
    :return: Stats for all rounds for a fighter as a list.
    """
    if len(stats) == 0:
        return [np.nan for _ in range(len(fighter_stats_keys))]
    res = {k: 0 for k in fighter_stats_keys}
    for i in stats:
        for k in res:
            res[k] = i.get(k, 0)
    return list(res.values())


def parse_fight_data(row: pd.Series) -> pd.Series:
    """
    Parse 'fighters' column.
    :param row: Row of the events dataframe.
    :return: pd.Series with stats for both fighters.
    """
    fighters = row["fighters"]
    if fighters == "[]" or fighters == np.nan:
        return pd.Series([np.nan for _ in range(len(fighter_stats_keys))])
    cols = []
    fighters = ast.literal_eval(fighters)
    if fighters[0]["fighterId"] == row["fighterId_2"]:
        fighters = reversed(fighters)
    for f in fighters:
        cols.extend(sum_round_stats(f["roundStats"]))
    return pd.Series(cols)

In [112]:
events_df[get_fighter_stats_cols()] = events_df[
    ["fighters", "fighterId_1", "fighterId_2"]
].apply(lambda row: parse_fight_data(row), axis=1)
events_df.drop(columns="fighters", inplace=True)
events_df.head(5)

,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,odds_f1,odds_f2,hitsTotal_f1,hitsSuccessful_f1,takedownTotal_f1,takedownSuccessful_f1,submissionAttempts_f1,takeovers_f1,accentedHitsTotal_f1,accentedHitsSuccessful_f1,knockdowns_f1,protectionPassage_f1,hitsHeadTotal_f1,hitsHeadSuccessful_f1,hitsBodyTotal_f1,hitsBodySuccessful_f1,hitsLegsTotal_f1,hitsLegsSuccessful_f1,accentedHitsPositionDistanceTotal_f1,accentedHitsPositionDistanceSuccessful_f1,accentedHitsPositionClinchTotal_f1,accentedHitsPositionClinchSuccessful_f1,accentedHitsPositionParterTotal_f1,accentedHitsPositionParterSuccessful_f1,hitsTotal_f2,hitsSuccessful_f2,takedownTotal_f2,takedownSuccessful_f2,submissionAttempts_f2,takeovers_f2,accentedHitsTotal_f2,accentedHitsSuccessful_f2,knockdowns_f2,protectionPassage_f2,hitsHeadTotal_f2,hitsHeadSuccessful_f2,hitsBodyTotal_f2,hitsBodySuccessful_f2,hitsLegsTotal_f2,hitsLegsSuccessful_f2,accentedHitsPositionDistanceTotal_f2,accentedHitsPositionDistanceSuccessful_f2,accentedHitsPositionClinchTotal_f2,accentedHitsPositionClinchSuccessful_f2,accentedHitsPositionParterTotal_f2,accentedHitsPositionParterSuccessful_f2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5201,Denver,USA,104.0,1993-11-12,Europe/Berlin,1646,1923,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,4.0,3.0,3.0,1.0,1.0,0.0,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5202,Denver,USA,52.0,1993-11-12,Europe/Berlin,1777,1883,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0,NaN,NaN,9.0,3.0,2.0,1.0,1.0,0.0,9.0,3.0,0.0,1.0,7.0,3.0,1.0,0.0,1.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,5.0,0.0,1.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0
5203,Denver,USA,59.0,1993-11-12,Europe/Berlin,1908,1923,UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,13.0,7.0,1.0,1.0,3.0,3.0,8.0,5.0,0.0,0.0,9.0,6.0
5204,Denver,USA,57.0,1993-11-12,Europe/Berlin,1631,1646,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5205,Denver,USA,138.0,1993-11-12,Europe/Berlin,1646,1924,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,7.0,4.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Добавляем данные о бойцах в датафрейм с боями

In [113]:
fighter_data_cols = fighters_df.drop(
    columns=["weightCategory.id", "weightCategory.name"]  # add "name"
).columns
events_df = events_df.join(
    fighters_df[fighter_data_cols], on="fighterId_1", rsuffix="_f1"
)
events_df = events_df.join(
    fighters_df[fighter_data_cols], on="fighterId_2", rsuffix="_f2"
)
events_df.head(5)

,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,odds_f1,odds_f2,hitsTotal_f1,hitsSuccessful_f1,takedownTotal_f1,takedownSuccessful_f1,submissionAttempts_f1,takeovers_f1,accentedHitsTotal_f1,accentedHitsSuccessful_f1,knockdowns_f1,protectionPassage_f1,hitsHeadTotal_f1,hitsHeadSuccessful_f1,hitsBodyTotal_f1,hitsBodySuccessful_f1,hitsLegsTotal_f1,hitsLegsSuccessful_f1,accentedHitsPositionDistanceTotal_f1,accentedHitsPositionDistanceSuccessful_f1,accentedHitsPositionClinchTotal_f1,accentedHitsPositionClinchSuccessful_f1,accentedHitsPositionParterTotal_f1,accentedHitsPositionParterSuccessful_f1,hitsTotal_f2,hitsSuccessful_f2,takedownTotal_f2,takedownSuccessful_f2,submissionAttempts_f2,takeovers_f2,accentedHitsTotal_f2,accentedHitsSuccessful_f2,knockdowns_f2,protectionPassage_f2,hitsHeadTotal_f2,hitsHeadSuccessful_f2,hitsBodyTotal_f2,hitsBodySuccessful_f2,hitsLegsTotal_f2,hitsLegsSuccessful_f2,accentedHitsPositionDistanceTotal_f2,accentedHitsPositionDistanceSuccessful_f2,accentedHitsPositionClinchTotal_f2,accentedHitsPositionClinchSuccessful_f2,accentedHitsPositionParterTotal_f2,accentedHitsPositionParterSuccessful_f2,name_f1,weight,height,armSpan,legSwing,dateOfBirth,birthYear,country_f1,city_f1,timezone_f1,name_f2,weight_f2,height_f2,armSpan_f2,legSwing_f2,dateOfBirth_f2,birthYear_f2,country_f2,city_f2,timezone_f2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5201,Denver,USA,104.0,1993-11-12,Europe/Berlin,1646,1923,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,4.0,3.0,3.0,1.0,1.0,0.0,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Royce Gracie,79.38,185.42,NaN,NaN,1966-12-12,1966.0,Brazil,Rio de Janeiro,America/Sao_Paulo,Gerard Gordeau,97.98,195.58,NaN,NaN,1959-03-30,1959.0,Netherlands,NaN,Europe/Amsterdam
5202,Denver,USA,52.0,1993-11-12,Europe/Berlin,1777,1883,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0,NaN,NaN,9.0,3.0,2.0,1.0,1.0,0.0,9.0,3.0,0.0,1.0,7.0,3.0,1.0,0.0,1.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,5.0,0.0,1.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,Jason DeLucia,86.18,180.34,NaN,NaN,1969-07-24,1969.0,United States,NaN,America/New_York,Trent Jenkins,83.91,187.96,NaN,NaN,1970-01-01,1970.0,United States,NaN,America/New_York
5203,Denver,USA,59.0,1993-11-12,Europe/Berlin,1908,1923,UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,13.0,7.0,1.0,1.0,3.0,3.0,8.0,5.0,0.0,0.0,9.0,6.0,Kevin Rosier,124.74,193.04,NaN,NaN,1970-01-01,1970.0,United States,NaN,America/New_York,Gerard Gordeau,97.98,195.58,NaN,NaN,1959-03-30,1959.0,Netherlands,NaN,Europe/Amsterdam
5204,Denver,USA,57.0,1993-11-12,Europe/Berlin,1631,1646,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Ken Shamrock,92.99,185.42,182.88,NaN,1964-02-11,1964.0,United States,Macon,America/New_York,Royce Gracie,79.38,185.42,NaN,NaN,1966-12-12,1966.0,Brazil,Rio de Janeiro,America/Sao_Paulo
5205,Denver,USA,138.0,1993-11-12,Europe/Berlin,1646,1924,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,7.0,4.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Royce Gracie,79.38,185.42,NaN,NaN,1966-12-12,1966.0,Brazil,Rio de Janeiro,America/Sao_Paulo,Art Jimmerson,88.90,185.42,NaN,NaN,1963-08-04,1963.0,United States,NaN,America/New_York


In [65]:
### Убрать выбросы в разделе Fighters
fighters_df.describe(include='all')

/opt/anaconda3/envs/datalore-user/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,name,weight,height,armSpan,legSwing,weightCategory.id,weightCategory.name,dateOfBirth,birthYear,country,city,timezone
count,3669,3530.000000,3335.000000,1727.000000,774.000000,3669.00000,3669,3599,3599.000000,2458,1361,2308
unique,3669,NaN,NaN,NaN,NaN,NaN,13,2418,NaN,152,907,129
top,Alfred Khashakyan,NaN,NaN,NaN,NaN,NaN,Средний вес,1970-01-01 00:00:00,NaN,United States,Rio de Janeiro,America/New_York
freq,1,NaN,NaN,NaN,NaN,NaN,853,761,NaN,1131,37,941
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1943-01-25 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999-09-13 00:00:00,NaN,NaN,NaN,NaN
mean,NaN,78.472513,178.680006,182.566925,101.829238,7.30199,NaN,NaN,1980.730203,NaN,NaN,NaN
std,NaN,18.186395,9.924185,12.444285,7.562750,3.12400,NaN,NaN,8.368212,NaN,NaN,NaN
min,NaN,47.630000,152.400000,152.400000,39.000000,2.00000,NaN,NaN,1943.000000,NaN,NaN,NaN
25%,NaN,65.770000,172.720000,175.260000,96.520000,5.00000,NaN,NaN,1971.000000,NaN,NaN,NaN


In [82]:
fighters_df_copy = fighters_df.copy()
while i < fighters_df_copy.height.
for i in fighters_df_copy.height.items():
    if i <= 205:
        print(i)


        
# for i in events_df:
#     if events_df[i].dtype != 'object':
#         print(f'В колонке {i} максимум {events_df[i].max()}')


TypeError: TypeError: '<=' not supported between instances of 'tuple' and 'int'

In [70]:
df = events_df.copy()

cols = ['odds',
 'hitsTotal',
 'hitsSuccessful',
 'takedownTotal',
 'takedownSuccessful',
 'submissionAttempts',
 'takeovers',
 'accentedHitsTotal',
 'accentedHitsSuccessful',
 'knockdowns',
 'protectionPassage',
 'hitsHeadTotal',
 'hitsHeadSuccessful',
 'hitsBodyTotal',
 'hitsBodySuccessful',
 'hitsLegsTotal',
 'hitsLegsSuccessful',
 'accentedHitsPositionDistanceTotal',
 'accentedHitsPositionDistanceSuccessful',
 'accentedHitsPositionClinchTotal',
 'accentedHitsPositionClinchSuccessful',
 'accentedHitsPositionParterTotal',
 'accentedHitsPositionParterSuccessful']

# если что, f1 и f2 теперь суффиксы, а не префиксы

def difference(df, cols):
    for col in cols:
        df[col] = df['f1_'+col] - df['f2_'+col]
        df.drop(['f1_'+col, 'f2_'+col], axis='columns', inplace=True)

        return df

In [71]:
df

,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_odds,f2_odds,f1_hitsTotal,f1_hitsSuccessful,f1_takedownTotal,f1_takedownSuccessful,f1_submissionAttempts,f1_takeovers,f1_accentedHitsTotal,f1_accentedHitsSuccessful,f1_knockdowns,f1_protectionPassage,f1_hitsHeadTotal,f1_hitsHeadSuccessful,f1_hitsBodyTotal,f1_hitsBodySuccessful,f1_hitsLegsTotal,f1_hitsLegsSuccessful,f1_accentedHitsPositionDistanceTotal,f1_accentedHitsPositionDistanceSuccessful,f1_accentedHitsPositionClinchTotal,f1_accentedHitsPositionClinchSuccessful,f1_accentedHitsPositionParterTotal,f1_accentedHitsPositionParterSuccessful,f2_hitsTotal,f2_hitsSuccessful,f2_takedownTotal,f2_takedownSuccessful,f2_submissionAttempts,f2_takeovers,f2_accentedHitsTotal,f2_accentedHitsSuccessful,f2_knockdowns,f2_protectionPassage,f2_hitsHeadTotal,f2_hitsHeadSuccessful,f2_hitsBodyTotal,f2_hitsBodySuccessful,f2_hitsLegsTotal,f2_hitsLegsSuccessful,f2_accentedHitsPositionDistanceTotal,f2_accentedHitsPositionDistanceSuccessful,f2_accentedHitsPositionClinchTotal,f2_accentedHitsPositionClinchSuccessful,f2_accentedHitsPositionParterTotal,f2_accentedHitsPositionParterSuccessful
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5201,Denver,USA,104.0,1993-11-12,Europe/Berlin,1646,1923,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,4.0,3.0,3.0,1.0,1.0,0.0,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5202,Denver,USA,52.0,1993-11-12,Europe/Berlin,1777,1883,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0,NaN,NaN,9.0,3.0,2.0,1.0,1.0,0.0,9.0,3.0,0.0,1.0,7.0,3.0,1.0,0.0,1.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,5.0,0.0,1.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0
5203,Denver,USA,59.0,1993-11-12,Europe/Berlin,1908,1923,UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,13.0,7.0,1.0,1.0,3.0,3.0,8.0,5.0,0.0,0.0,9.0,6.0
5204,Denver,USA,57.0,1993-11-12,Europe/Berlin,1631,1646,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5205,Denver,USA,138.0,1993-11-12,Europe/Berlin,1646,1924,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,7.0,4.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40497,Las Vegas,USA,300.0,2021-02-27,Europe/Berlin,421,668,UFC Fight Night,3.0,America/Los_Angeles,5,Легкий вес,['DEC'],668.0,1.46,2.84,63.0,17.0,0.0,0.0,0.0,0.0,63.0,17.0,0.0,0.0,45.0,6.0,10.0,3.0,8.0,8.0,63.0,17.0,0.0,0.0,0.0,0.0,71.0,16.0,0.0,0.0,0.0,0.0,71.0,16.0,0.0,0.0,62.0,12.0,3.0,1.0,6.0,3.0,71.0,16.0,0.0,0.0,0.0,0.0
40498,Las Vegas,USA,158.0,2021-02-27,Europe/Berlin,3504,3521,UFC Fight Night,3.0,America/Los_Angeles,3,Легчайший вес,['KO'],3504.0,1.56,2.53,90.0,72.0,1.0,1.0,0.0,2.0,68.0,50.0,0.0,0.0,64.0,47.0,3.0,2.0,1.0,1.0,3.0,2.0,0.0,0.0,65.0,48.0,4.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
40500,Las Vegas,USA,300.0,2021-02-27,Europe/Berlin,1334,3463,UFC Fight Night,3.0,America/Los_Angeles,8,Полутяжелый вес,['DEC'],1334.0,1.63,2.37,61.0,29.0,0.0,0.0,0.0,0.0,57.0,26.0,0.0,0.0,40.0,15.0,3.0,2.0,14.0,9.0,57.0,26.0,0.0,0.0,0.0,0.0,78.0,30.0,0.0,0.0,0.0,0.0,75.0,27.0,0.0,0.0,56.0,12.0,4.0,2.0,15.0,13.0

In [63]:
difference(df, cols)

,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_hitsTotal,f1_hitsSuccessful,f1_takedownTotal,f1_takedownSuccessful,f1_submissionAttempts,f1_takeovers,f1_accentedHitsTotal,f1_accentedHitsSuccessful,f1_knockdowns,f1_protectionPassage,f1_hitsHeadTotal,f1_hitsHeadSuccessful,f1_hitsBodyTotal,f1_hitsBodySuccessful,f1_hitsLegsTotal,f1_hitsLegsSuccessful,f1_accentedHitsPositionDistanceTotal,f1_accentedHitsPositionDistanceSuccessful,f1_accentedHitsPositionClinchTotal,f1_accentedHitsPositionClinchSuccessful,f1_accentedHitsPositionParterTotal,f1_accentedHitsPositionParterSuccessful,f2_hitsTotal,f2_hitsSuccessful,f2_takedownTotal,f2_takedownSuccessful,f2_submissionAttempts,f2_takeovers,f2_accentedHitsTotal,f2_accentedHitsSuccessful,f2_knockdowns,f2_protectionPassage,f2_hitsHeadTotal,f2_hitsHeadSuccessful,f2_hitsBodyTotal,f2_hitsBodySuccessful,f2_hitsLegsTotal,f2_hitsLegsSuccessful,f2_accentedHitsPositionDistanceTotal,f2_accentedHitsPositionDistanceSuccessful,f2_accentedHitsPositionClinchTotal,f2_accentedHitsPositionClinchSuccessful,f2_accentedHitsPositionParterTotal,f2_accentedHitsPositionParterSuccessful,odds
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5201,Denver,USA,104.0,1993-11-12,Europe/Berlin,1646,1923,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,4.0,3.0,3.0,1.0,1.0,0.0,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5202,Denver,USA,52.0,1993-11-12,Europe/Berlin,1777,1883,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0,9.0,3.0,2.0,1.0,1.0,0.0,9.0,3.0,0.0,1.0,7.0,3.0,1.0,0.0,1.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,5.0,0.0,1.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,NaN
5203,Denver,USA,59.0,1993-11-12,Europe/Berlin,1908,1923,UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,13.0,7.0,1.0,1.0,3.0,3.0,8.0,5.0,0.0,0.0,9.0,6.0,NaN
5204,Denver,USA,57.0,1993-11-12,Europe/Berlin,1631,1646,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5205,Denver,USA,138.0,1993-11-12,Europe/Berlin,1646,1924,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,7.0,4.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40497,Las Vegas,USA,300.0,2021-02-27,Europe/Berlin,421,668,UFC Fight Night,3.0,America/Los_Angeles,5,Легкий вес,['DEC'],668.0,63.0,17.0,0.0,0.0,0.0,0.0,63.0,17.0,0.0,0.0,45.0,6.0,10.0,3.0,8.0,8.0,63.0,17.0,0.0,0.0,0.0,0.0,71.0,16.0,0.0,0.0,0.0,0.0,71.0,16.0,0.0,0.0,62.0,12.0,3.0,1.0,6.0,3.0,71.0,16.0,0.0,0.0,0.0,0.0,-1.38
40498,Las Vegas,USA,158.0,2021-02-27,Europe/Berlin,3504,3521,UFC Fight Night,3.0,America/Los_Angeles,3,Легчайший вес,['KO'],3504.0,90.0,72.0,1.0,1.0,0.0,2.0,68.0,50.0,0.0,0.0,64.0,47.0,3.0,2.0,1.0,1.0,3.0,2.0,0.0,0.0,65.0,48.0,4.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,-0.97
40500,Las Vegas,USA,300.0,2021-02-27,Europe/Berlin,1334,3463,UFC Fight Night,3.0,America/Los_Angeles,8,Полутяжелый вес,['DEC'],1334.0,61.0,29.0,0.0,0.0,0.0,0.0,57.0,26.0,0.0,0.0,40.0,15.0,3.0,2.0,14.0,9.0,57.0,26.0,0.0,0.0,0.0,0.0,78.0,30.0,0.0,0.0,0.0,0.0,75.0,27.0,0.0,0.0,56.0,12.0,4.0,2.0,15.0,13.0,75.0,27.0,0.0,0.0,0.0,0.0,-0.74


In [67]:
for i in cols:
    print('f1_' + i)

f1_odds
f1_hitsTotal
f1_hitsSuccessful
f1_takedownTotal
f1_takedownSuccessful
f1_submissionAttempts
f1_takeovers
f1_accentedHitsTotal
f1_accentedHitsSuccessful
f1_knockdowns
f1_protectionPassage
f1_hitsHeadTotal
f1_hitsHeadSuccessful
f1_hitsBodyTotal
f1_hitsBodySuccessful
f1_hitsLegsTotal
f1_hitsLegsSuccessful
f1_accentedHitsPositionDistanceTotal
f1_accentedHitsPositionDistanceSuccessful
f1_accentedHitsPositionClinchTotal
f1_accentedHitsPositionClinchSuccessful
f1_accentedHitsPositionParterTotal
f1_accentedHitsPositionParterSuccessful
